In [8]:
%pip install transformers datasets
from transformers import pipeline
from datasets import load_dataset
import pandas as pd
import torch

# Load the dataset
dataset = load_dataset('yelp_review_full')
df = dataset['train'].to_pandas()  

# Create a text summarization pipeline using the BART model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Initialize an empty list to store summaries
summaries = []

# Dynamically adjust max_length and summarize the text
for text in df['text'].iloc[:150].tolist():
    input_length = len(text.split())  # Estimate the number of words in the input
    max_length = min(130, max(30, int(input_length * 0.5)))  # Dynamically adjust max_length
    summary = summarizer(text, max_length=max_length, min_length=30, truncation=True)[0]['summary_text']
    summaries.append(summary)

# Create a new dataframe to store the original texts and their summaries
# Make sure to also copy the star rating information, assuming in the original dataset the star rating is stored in the 'label' column
df_summary = df.iloc[:150].copy()  # Copy the first 150 rows
df_summary['summary'] = summaries  # Add the summaries to the new dataframe
df_summary['stars'] = df['label'].iloc[:150]  # Copy the corresponding star ratings

print(df_summary[['text', 'summary', 'stars']])


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


Your max_length is set to 30, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 30, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max_length is set to 30, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max_length is set to 30, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_leng

                                                  text  \
0    dr. goldberg offers everything i look for in a...   
1    Unfortunately, the frustration of being Dr. Go...   
2    Been going to Dr. Goldberg for over 10 years. ...   
3    Got a letter in the mail last week that said D...   
4    I don't know what Dr. Goldberg was like before...   
..                                                 ...   
145  Even when we didn't have a car Filene's Baseme...   
146  Love this store!  Don't always have much luck ...   
147  Another store which has gone the way of the Do...   
148  $9.75 for a red bull and vodka? I'm sorry, I t...   
149  Really enjoyed this a lot more than I thought ...   

                                               summary  stars  
0     dr. goldberg offers everything i look for in ...      4  
1    The frustration of being Dr. Goldberg's patien...      1  
2    I've been going to Dr. Goldberg for over 10 ye...      3  
3    Dr. Goldberg is moving to Arizona to take 

Data Preparation

In [9]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import numpy as np
import torch

class YelpReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels - 1  # Convert labels from 1-5 to 0-4

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# Prepare the data
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
X = df_summary['summary'].tolist()  # Summary texts
y = df_summary['stars'].to_numpy()  # Star ratings

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2021)

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=512)

train_dataset = YelpReviewDataset(train_encodings, y_train)
val_dataset = YelpReviewDataset(val_encodings, y_val)


Model Definition, Training and Evaluation.

In [10]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()
trainer.evaluate()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/45 [00:00<?, ?it/s]

{'loss': 1.2102, 'grad_norm': 5.436811923980713, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.67}
{'loss': 1.2112, 'grad_norm': 5.261388301849365, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.33}
{'loss': 1.1035, 'grad_norm': 5.03201150894165, 'learning_rate': 3e-06, 'epoch': 2.0}
{'loss': 1.143, 'grad_norm': 5.402960777282715, 'learning_rate': 4.000000000000001e-06, 'epoch': 2.67}
{'train_runtime': 23.2161, 'train_samples_per_second': 15.506, 'train_steps_per_second': 1.938, 'train_loss': 1.1458648787604437, 'epoch': 3.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 1.2915648221969604,
 'eval_runtime': 0.5079,
 'eval_samples_per_second': 59.072,
 'eval_steps_per_second': 7.876,
 'epoch': 3.0}


# Result List
5   'train_loss': 1.1762760480244954 'eval_loss': 1.5997843742370605

15  'train_loss': 1.3541520436604817 'eval_loss': 0.7361515164375305

30  'train_loss': 1.2990621990627713 'eval_loss': 1.175505518913269

100 'train_loss': 1.2386985460917155 'eval_loss': 1.1782324314117432

150 'train_loss': 1.1339557965596516 'eval_loss': 1.2087987661361694

The second result in the list shows an intriguing case where the evaluation loss is significantly lower than the training loss. This is an unusual scenario in machine learning models because typically, the training loss is lower due to the model being directly trained on that data. Here, an eval_loss of 0.736 compared to a train_loss of 1.354 suggests a few possible scenarios:

Strong Generalization: The model may generalize exceptionally well to the validation data, possibly because the validation set characteristics closely match the model's training data, or the model has learned robust features that work well across varied data samples.

Data Distribution: The distribution of data in the training set might be more challenging or diverse compared to the evaluation set, which can lead to higher training loss.

Evaluation Set Size: If the evaluation set is significantly smaller or not as diverse, it might not represent the difficulty of the overall dataset, leading to a deceptively low evaluation loss.

Overfitting Avoidance: It's also possible that certain regularization techniques or early stopping mechanisms were effective in preventing the model from overfitting, hence the model performs better on unseen data.

Random Variation: Especially with smaller datasets, random variation can sometimes lead to situations where evaluation loss is lower than training loss.

# The number 300 is used to specify the number of text entries from the Yelp review dataset that we want to process.

1.Data Selection: df['text'].iloc[:300].tolist() – This line selects the first 300 entries from the 'text' column of the dataframe df. The .iloc[:300] is an indexing operation used in pandas to select data by position. Here, it slices the first 300 rows of the dataframe.

2.Summarization Process: The loop for text in df['text'].iloc[:300].tolist() iterates over each of the first 300 text entries. For each text, the script estimates the input length, adjusts the maximum summary length accordingly, and then generates a summary using the BART model.

# Ideal Range

Close to 0 but not 0: Ideally, the loss value should be close to 0, which indicates that the model's predictions are very close to the actual labels. However, a loss value of 0 might suggest that the model is overfitting on the training set, perfectly memorizing the training data, which could lead to poor performance on unseen data.

Eval loss close to train loss: If the evaluation loss (eval_loss) and training loss (train_loss) are very close, it is usually a good sign, indicating that the model performs consistently on both the training and validation sets, demonstrating good generalization ability.

# Specific Range

Loss value: The specific range of loss values greatly depends on the characteristics of the dataset and the complexity of the model. For some tasks, such as simple classification problems, the loss value might be between 0.01 and 0.5. For more complex tasks or more challenging datasets, the loss value might range from 1 to 10, or even higher.

Difference: The difference between the training loss and evaluation loss should be as small as possible. A good rule of thumb is that the difference should not exceed 10% to 20% of the training loss. For example, if the train_loss is 0.5, the eval_loss should ideally be between 0.5 and 0.6.

# Loss Values in Our Model

Training Loss (train_loss) of 1.1339557965596516: This indicates the average degree of error the model has when recognizing patterns in the training data. For a complex NLP task, this value might be considered reasonable in the initial stages, especially considering the use of a large pre-trained model and a complex dataset.

Evaluation Loss (eval_loss) of 1.2087987661361694: This is slightly higher than the training loss. Such a scenario is common in machine learning models, as the evaluation loss represents the model's performance on unseen data, which is usually slightly higher than the training loss.

# Judging Whether Loss Values are "High" or "Low"

Generally, for text classification tasks, if both training and evaluation losses are below 1.0, this is usually considered good performance. However, even if loss values are between 1.0 and 2.0, this may still be acceptable performance depending on the task's complexity and the characteristics of the dataset.

It's important to monitor how loss values change with training progress. Ideally, you would want to see both training and evaluation losses gradually decrease as the number of epochs increases, and the gap between them should not significantly widen over time, as this may indicate overfitting.